In [13]:
%run ForestPipeline.ipynb

In [14]:
import param as pm
import panel as pn
import pandas as pd
import datetime as dt
import numpy as np
import hvplot.pandas
pn.extension()

Revenue Share Per Acre

In [15]:
f1.centree_commission_per_hectare()

30.32629227868413

Minting Fee per Acre

In [16]:
f1.centree_mint_fee_per_hectare()

19.83963046269056

Administration Fee Per Acre Per Month

In [17]:
c1.compute()['Centree (Admin/Commission)'].iloc[0] / f1.forested_area_hectares / 12

0.3558794979899372

Forested Area Hectares

In [18]:
f1.forested_area_hectares

7478

Centree Commission per Hectare

In [19]:
f1.centree_commission_per_hectare()

30.32629227868413

Centree Mint Fee per Hectare

In [20]:
f1.centree_mint_fee_per_hectare()

19.83963046269056

In [25]:
class Web3Pilot(pm.Parameterized):
    
    def __init__(self, forest_pipeline: list, **params):
        super(Web3Pilot, self).__init__(**params)
        self.forest_pipeline = forest_pipeline
        
    def cumulative_revenue(self, periods=24, freq='M'):
        df = pd.DataFrame(index=pd.date_range(dt.datetime.now(), periods=periods, freq=freq))
        df['hectares_protected'] = 0
        df['Revenue Share'] = 0
        df['Minting Fee'] = 0
        df['On-going'] = 0
        for project in self.forest_pipeline.projects:
            df.at[df.index[project.month], 'hectares_protected'] += project.forest.forested_area_hectares
            df.at[df.index[project.month], 'Revenue Share'] += project.forest.forested_area_hectares * project.forest.centree_commission_per_hectare()
            df.at[df.index[project.month], 'Minting Fee'] += project.forest.forested_area_hectares * project.forest.centree_mint_fee_per_hectare()
        df['hectares_protected_cumul'] = df['hectares_protected'].cumsum()
        for forest in self.forest_pipeline.projects:
            df.loc[df.index[project.month]:, 'On-going'] += project.forest.forested_area_hectares * project.carbon.compute()['Centree (Admin/Commission)'].iloc[0] / project.forest.forested_area_hectares / 12
        df['Total Revenue - Cash'] = df['Revenue Share'] + df['Minting Fee'] + df['On-going']
        df['Cumulative Revenue - Cash'] = df['Total Revenue - Cash'].cumsum()
        df.index.name = 'Date'
        return df
    
    def view_cumulative_revenue_table(self):
        return self.cumulative_revenue().reset_index().hvplot.table(height=800,width=800)
    
    def view_cumulative_revenue_chart(self):
        return self.cumulative_revenue().hvplot(y='Cumulative Revenue - Cash', title='Web3 Pilot Cumulative Revenue')

In [26]:
w = Web3Pilot(forest_pipeline)

In [27]:
w.view_cumulative_revenue_chart()

:Curve   [Date]   (Cumulative Revenue - Cash)

In [28]:
w.view_cumulative_revenue_table()

:Table   [Date,hectares_protected,Revenue Share,Minting Fee,On-going,hectares_protected_cumul,Total Revenue - Cash,Cumulative Revenue - Cash]